In [1]:
import datetime
import math
import time
import itertools
from contextlib import closing
from collections import OrderedDict
import json
import numpy as np
import pandas as pd
import MySQLdb
import pymssql
from sqlalchemy import create_engine
import dfutils

In [2]:
def optimized_execute(query, commit=False):
    '''
    Executes the search query in Northpoint
    '''
    
    with closing(pymssql.connect(host='10.16.1.16', user='readonly_user', password='readonly_user',
                                 database='PnLAppDb')) as np_pnl_conn:
        with closing(np_pnl_conn.cursor()) as np_pnl_cursor:
            np_pnl_cursor.execute(query)
            if commit:
                np_pnl_conn.commit()
                return

            fetched_res = np_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
            return fetched_res

In [3]:
def get_position_calculated_values_history_max_date():  # done
    query = "SELECT MAX(PCVH.TradeDate) FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH "
    results = optimized_execute(query)
    if len(results) == 0:
        return None

    return results[0][0]

In [4]:
    def get_position_calculated_values_history(start_date_yyyy_mm_dd=None, end_date_yyyy_mm_dd=None,
                                               limit_to_tradegroups=[],
                                               is_tg_names_in_tradar_format=False, fund_code=None, NP_qty_rollup=True):
        if start_date_yyyy_mm_dd is None or end_date_yyyy_mm_dd is None:
            start_date_yyyy_mm_dd = '2017-01-01'
            end_date_yyyy_mm_dd = northpoint.get_position_calculated_values_history_max_date()  # end_date_yyyy_mm_dd = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        tg_filter_clause = ""
        if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
            tg_filter_clause = " AND PCVH.TradeGroup IN (" + ",".join(
                [("'" + tg + "'") for tg in limit_to_tradegroups if tg is not None]) + ") "
            if is_tg_names_in_tradar_format: tg_filter_clause = tg_filter_clause.replace('PCVH.TradeGroup',
                                                                                         'SUBSTRING(PCVH.TradeGroup,0,21)')
        fund_code_filtuer_clause = " " if fund_code is None else " AND PCVH.FundCode = '" + fund_code + "' "
        # region query
        # New Fund by added bv Kshitij "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL":"MACO"
        query = "SELECT " \
                "PCVH.TradeDate, " \
                "ISNULL(PCVH.FundCode, " \
                "CASE F.FundName " \
                "WHEN 'Columbia' THEN 'CAM' " \
                "WHEN 'Litman Gregory' THEN 'LG' " \
                "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
                "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
                "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
                "WHEN 'TransAmerica' THEN 'TAF' " \
                "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
                "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
                "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
                "WHEN 'Water Island Capital Lev Arb Fund' THEN 'WED' " \
                "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
                "END " \
                ") AS FundCode, " \
                "SPT_TRDGRP.SecName AS TradeGroup, " \
                "SPT_TRDGRP.SecName AS TradeGroup_Tradar_Name, " \
                "PCVH.[Marketing GROUP] AS Sleeve,   " \
                "ISNULL(PCVH.[Bucket],'NA') AS Bucket, " \
                "PCVH.TradeGroupId, " \
                "SPT.Ticker, " \
                "SPT.BloombergGlobalId, " \
                "SPT.BloombergGID, " \
                "ISNULL(SPT.BloombergGlobalId,SPT.BloombergGID) AS BBGID, " \
                "SPT.SecType, " \
                "ISNULL(SPT.MarketCapCategory, 'N/A') AS MarketCapCategory, " \
                "PCVH.SecurityID, " \
                "PCVH.DealTermsCash, " \
                "PCVH.DealTermsStock, " \
                "PCVH.DealValue, " \
                "PCVH.DealClosingDate, " \
                "CASE " \
                "WHEN SPT.SecType = 'FxFwd' THEN 0.0 " \
                "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Equity Special Situations'  THEN 1.0 " \
                "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Opportunistic'  THEN 1.0 " \
                "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Merger Arbitrage'  THEN " \
                "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') THEN " \
                "CASE PCVH.DealValue WHEN 0 THEN 0.0 ELSE PCVH.DealTermsStock/PCVH.DealValue END " \
                "ELSE 1.0 END " \
                "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Credit Opportunities'  THEN " \
                "CASE WHEN SPT.SecType IN ('EQ','ExchOpt') THEN 1.0 ELSE 0.0 END " \
                "ELSE NULL END " \
                "AS [Equity Risk Factor], " \
                "PCVH.DV_01, " \
                "PCVH.CR_01, " \
                "PCVH.Adj_CR_01, " \
                "ISNULL(SPT.UltimateCountry,'N/A') AS UltimateCountry, " \
                "ISNULL(PCVH.AlphaHedge,'NA') AS AlphaHedge, " \
                "SUM(PCVH.ExposureLong_USD) AS Exposure_Long, " \
                "SUM(PCVH.ExposureShort_USD) AS Exposure_Short, " \
                "SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD) AS Exposure_Net, " \
                "CASE WHEN NAV.NAV = 0 THEN NULL ELSE 100.0*(SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD)/NAV.NAV) END AS [Exposure_Net(%)], " \
                "SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD) AS NetMktVal, " \
                "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') THEN ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD)) ELSE NULL END AS Capital, " \
                "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') THEN 100.0*(ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD))/NAV.NAV) ELSE NULL END AS [Capital % of NAV], " \
                "100.0*SUM(PCVH.Base_Case_NAV_Impact) AS BaseCaseNavImpact, " \
                "100.0*SUM(PCVH.Outlier_NAV_Impact) AS OutlierNavImpact, " \
                "SUM(PCVH.Qty) AS QTY, " \
                "ISNULL(PCVH.TradeGroupLongShortFlag,'NA') AS LongShort, " \
                "ISNULL(SPT.GICSSectorName,'NA') AS Sector, " \
                "ISNULL(SPT.GICSIndustryName,'NA') AS Industry, " \
                "ISNULL(PCVH.TradeGroupCatalystRating, 'NA') AS CatalystRating, " \
                "CASE ISNULL(PCVH.TradeGroupCatalystRating, 'NA') " \
                "   WHEN 'NA' THEN 'N/A' " \
                "   WHEN 0 THEN 'No Catalyst' " \
                "   WHEN 1 THEN 'Soft' " \
                "   WHEN 2 THEN 'Binary' " \
                "   WHEN 3 THEN 'Hard' " \
                "   ELSE 'Unknown' END AS CatalystName, " \
                "NAV.NAV, " \
                "PCVH.Analyst " \
                "FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH " \
                "INNER JOIN PnLAppDb.dbo.Funds AS F ON PCVH.Portfolio = F.FundID " \
                "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON PCVH.SecurityID = SPT.ID " \
                "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
                "LEFT OUTER JOIN PnLAppDb.pnl.DailyNAV AS NAV ON PCVH.Portfolio = NAV.FundId AND PCVH.TradeDate = NAV.[DATE] " \
                "WHERE  TradeDate >= '" + start_date_yyyy_mm_dd + "' AND TradeDate <= '" + end_date_yyyy_mm_dd + "' " \
                + tg_filter_clause + \
                fund_code_filtuer_clause + \
                "GROUP BY " \
                "F.FundName, PCVH.FundCode,PCVH.TradeDate, PCVH.[Marketing GROUP], SPT_TRDGRP.SecName, " \
                "PCVH.TradeGroupId, SPT.Ticker, SPT.BloombergGlobalId, SPT.BloombergGID, SPT.SecType, ISNULL(SPT.MarketCapCategory, 'N/A'), " \
                "PCVH.SecurityId, SPT.UltimateCountry, " \
                "ISNULL(PCVH.AlphaHedge,'NA'),PCVH.TradeGroupLongShortFlag, ISNULL(SPT.GICSSectorName,'NA'), " \
                "ISNULL(SPT.GICSIndustryName,'NA'), ISNULL(PCVH.TradeGroupCatalystRating, 'NA'), " \
                "ISNULL(PCVH.[Bucket],'NA'), PCVH.DealTermsCash, PCVH.DealTermsStock, PCVH.DealValue, PCVH.DealClosingDate, " \
                "PCVH.DV_01, PCVH.CR_01, PCVH.Adj_CR_01, NAV.NAV, PCVH.Analyst " \
                "ORDER BY PCVH.TradeDate "
        # endregion
        results = optimized_execute(query)
        df = pd.DataFrame(results,
                          columns=["Date", "FundCode", "TradeGroup", "TradeGroup_Tradar_Name", "Sleeve", "Bucket",
                                   "TradeGroupId", "Ticker", "BloombergGlobalId", "BloombergGID", "BBGID",
                                   "SecType", "MarketCapCategory", "SecurityId", "DealTermsCash",
                                   "DealTermsStock", "DealValue", "DealClosingDate", "Equity_Risk_Factor",
                                   "DV01", "CR01", "Adj_CR01", "UltimateCountry",
                                   "AlphaHedge", "Exposure_Long", "Exposure_Short", "Exposure_Net",
                                   "Exposure_Net(%)", "NetMktVal",
                                   "Capital($)", "Capital(%)", "BaseCaseNavImpact", "OutlierNavImpact", "Qty",
                                   "LongShort", "Sector", "Industry", "CatalystRating", "CatalystName",
                                   "Fund_NAV", 'Analyst'
                                   ])

        float_cols = ['NetMktVal', 'Capital($)', 'Capital(%)', 'Exposure_Net', 'Exposure_Net(%)', 'BaseCaseNavImpact']
        df[float_cols] = df[float_cols].astype(float)
        df['Equity_Risk_Exp'] = df['Equity_Risk_Factor'].astype(float) * df['Exposure_Net'].astype(float)
        df[['DV01', 'CR01', 'Adj_CR01']] = df[['DV01', 'CR01', 'Adj_CR01']].fillna(0).astype(float)
        df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))  # convert timestamp to period. convention
        df["DealClosingDate"] = df["DealClosingDate"].apply(
            lambda x: pd.to_datetime(x))  # convert timestamp to period. convention

        as_of_dt = datetime.datetime.strptime(end_date_yyyy_mm_dd, '%Y-%m-%d')

        def duration_calc(days):
            if days <= 90: return '0M-3M'
            if days <= 180: return '3M-6M'
            if days <= 365: return '6M-12M'
            return 'Yr+'

        df["DealDuration"] = df["DealClosingDate"].apply(
            lambda x: None if pd.isnull(x) else duration_calc((as_of_dt - pd.to_datetime(x)).days))

        # region ADJUSTING FOR POSTED TRADES AFTER TRADE DATE - AFFECTING QTY ONLY
        # adjust for end_date_yyyy_mm_dd - take trades with tradedate = end_date_yyyy_mm_dd and posted_date AFTER end_date_yyyy_mm_dd.
        # adjust the Qty into the pcvh_df
        if NP_qty_rollup:
            qty_adjust_query = "SELECT " \
                               "A.TradeDate AS [DATE], " \
                               "CASE A.Fund " \
                               "WHEN 'Columbia' THEN 'CAM' " \
                               "WHEN 'Litman Gregory' THEN 'LG' " \
                               "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
                               "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
                               "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
                               "WHEN 'TransAmerica' THEN 'TAF' " \
                               "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
                               "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
                               "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
                               "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
                               "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL COMMINGLED MASTER FUND LP' THEN 'MACO' " \
                               "END AS FundCode, " \
                               "A.TradeGroupId, " \
                               "A.SecurityId, " \
                               "SUM((CASE SPT.SecType WHEN 'ExchOpt' THEN 100.0 ELSE 1.0 END)*A.Shares) AS Qty " \
                               "FROM " \
                               "[PnLAppDb].[dbo].[vTradesFlatView] AS A " \
                               "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON A.SecurityId = SPT.ID " \
                               "WHERE PostedDate >= DATEADD(DAY,1,TradeDate) AND TradeDate >= '" + start_date_yyyy_mm_dd + "' " \
                                                                                                                           "GROUP BY A.TradeDate, A.SecurityId, A.Ticker, A.Fund, A.TradeGroupId, SPT.SecType "
            results = optimized_execute(qty_adjust_query)
            qty_adjust_df = pd.DataFrame(results, columns=['Date', 'FundCode', 'TradeGroupId', 'SecurityId', 'Qty_adj'])
            qty_adjust_df[['TradeGroupId', 'SecurityId', 'Qty_adj']] = qty_adjust_df[
                ['TradeGroupId', 'SecurityId', 'Qty_adj']].astype(float)
            qty_adjust_df['Date'] = qty_adjust_df['Date'].apply(lambda x: pd.to_datetime(x))

            df[['TradeGroupId', 'SecurityId', 'Qty']] = df[['TradeGroupId', 'SecurityId', 'Qty']].astype(float)
            df = pd.merge(df, qty_adjust_df, how='left', on=['Date', 'FundCode', 'TradeGroupId', 'SecurityId'])
            df['Qty'] = df['Qty'] + df['Qty_adj'].fillna(0)
            del df['Qty_adj']
        # endregion
        print('PCVH retrieval completed....')
        return df

In [5]:
#def get_capital_allocation(as_of_yyyy_mm_dd=None, metric="Capital(%)"):
start = time.time()
metric="Capital(%)"
as_of_yyyy_mm_dd = None
if as_of_yyyy_mm_dd is None:
    as_of_yyyy_mm_dd = get_position_calculated_values_history_max_date()

if datetime.datetime.strptime(as_of_yyyy_mm_dd,'%Y-%m-%d') < datetime.datetime(2015,11,9,0,0):
    #return "<h9>Function not supported for dates prior to 11/09/2015</h9>"
    print('no dates prior to 2015')

pcvh_df = get_position_calculated_values_history(as_of_yyyy_mm_dd,as_of_yyyy_mm_dd)
pcvh_df = pcvh_df[pcvh_df['Sleeve'] != 'Forwards']

    #timestamp, risk_df = dbutils.wic.get_risk_snapshot()
    #risk_df = risk_df[risk_df['Sleeve'] != 'Forwards']
sleeves = list(pcvh_df['Sleeve'].dropna().unique())
fund_codes = list(pcvh_df['FundCode'].dropna().unique())
#level_one_tabs = []
total_sleeves = pd.DataFrame()
capital_allocation_slv_dict = {}
for slv in sleeves:
    slv_pcvh_df = pcvh_df[pcvh_df['Sleeve']==slv].copy()

    if len(slv_pcvh_df) == 0: continue

    buckets = slv_pcvh_df['Bucket'].dropna().unique()
    if len(buckets) == 0: continue
      
    tot_slv_df = pd.DataFrame()
    all_buckets = pd.DataFrame()
    capital_allocation_bkt_dict = []
    for bucket in buckets:
        bucket_slv_pcvh_df = slv_pcvh_df[slv_pcvh_df['Bucket']==bucket].copy()
        df = bucket_slv_pcvh_df[['Bucket', 'TradeGroup','LongShort','CatalystName']].drop_duplicates()
        capital_allocation_tg_dict = []
        for fund_code in fund_codes:
            fund_capital_cln_df = bucket_slv_pcvh_df[bucket_slv_pcvh_df[
                'FundCode']==fund_code].groupby(['Bucket', 'TradeGroup','LongShort','CatalystName']
                                               ).sum().reset_index()[['Bucket', 'TradeGroup', 'LongShort',
                                                                        'CatalystName',
                                                                        metric]].rename(columns={metric:fund_code})
            df = pd.merge(df,fund_capital_cln_df, how='left', on=['Bucket', 'TradeGroup', 'LongShort',
                                                                   'CatalystName']) 
        reduced_df = df
        tradegroups = reduced_df['TradeGroup']
        reduced_df.drop(columns=['Bucket', 'TradeGroup'])
        for i in range(0,len(tradegroups)):
            capital_allocation_tg_dict.append({'TradeGroup': tradegroups[i], 'FundData': reduced_df.iloc[i].to_dict()})

        capital_allocation_bkt_dict.append({'Bucket': bucket, 'Subdata': capital_allocation_tg_dict}) 
        tot_slv_df = tot_slv_df.append(df.groupby(['Bucket']).sum().reset_index()[['Bucket']+fund_codes])
    tot_slv_df['Sleeve'] = slv
    capital_allocation_slv_dict[slv] = capital_allocation_bkt_dict    
    total_sleeves = total_sleeves.append(tot_slv_df)

    tot_slv_df.index = range(len(tot_slv_df)) # appending messes up index
    tot_slv_df = tot_slv_df.dropna(axis=1, how='all')

    bucket2fund2metric = {bucket:{fund:(float(tot_slv_df[tot_slv_df['Bucket']
                                                             ==bucket][fund].fillna(0).iloc[0]) if 
                                            fund in tot_slv_df.columns else 0.0) for fund in fund_codes } 
                              for bucket in tot_slv_df['Bucket']}







print(time.time()-start)

PCVH retrieval completed....
4.382999897


In [6]:
capital_allocation_slv_dict

{u'Credit Opportunities': [{'Bucket': u'Catalyst-Driven Credit',
   'Subdata': [{'FundData': {u'AED': 1.8203,
      u'ARB': nan,
      'Bucket': u'Catalyst-Driven Credit',
      u'CAM': 1.7037,
      'CatalystName': u'Soft',
      u'LEV': nan,
      u'LG': 1.7912,
      'LongShort': u'Long',
      u'MACO': nan,
      u'MALT': nan,
      u'TACO': 3.2524,
      u'TAQ': nan,
      'TradeGroup': u'NLSN SPEC M&A',
      u'WED': nan,
      u'WIC': nan},
     'TradeGroup': u'NLSN SPEC M&A'},
    {'FundData': {u'AED': 2.1365,
      u'ARB': nan,
      'Bucket': u'Catalyst-Driven Credit',
      u'CAM': 2.0381,
      'CatalystName': u'Soft',
      u'LEV': nan,
      u'LG': 2.5469,
      'LongShort': u'Long',
      u'MACO': nan,
      u'MALT': nan,
      u'TACO': 3.5886,
      u'TAQ': nan,
      'TradeGroup': u'QEP YTC/SPEC M&A BONDS',
      u'WED': nan,
      u'WIC': nan},
     'TradeGroup': u'QEP YTC/SPEC M&A BONDS'},
    {'FundData': {u'AED': 1.0457,
      u'ARB': nan,
      'Bucket': u'Catalys

In [100]:
total_sleeves = total_sleeves.set_index(['Sleeve', 'Bucket'])
unique_sleeves = total_sleeves.index.get_level_values('Sleeve').unique()

In [103]:
chart_dict = {}
for i in unique_sleeves:
    chart_dict[i] = total_sleeves.xs(i).to_dict()

In [105]:
chart_dict

{u'Credit Opportunities': {u'AED': {u'Catalyst-Driven Credit': 12.853399999999999,
   u'Deep Value': 0.0,
   u'Merger Related Credit': 8.991499999999998,
   u'Relative Value Credit': 0.0,
   u'Yield to Call': 0.7537},
  u'ARB': {u'Catalyst-Driven Credit': 0.0,
   u'Deep Value': 0.0,
   u'Merger Related Credit': 1.9017000000000002,
   u'Relative Value Credit': 0.0,
   u'Yield to Call': 0.0},
  u'CAM': {u'Catalyst-Driven Credit': 12.0307,
   u'Deep Value': 0.0,
   u'Merger Related Credit': 8.3825,
   u'Relative Value Credit': 0.0,
   u'Yield to Call': 0.7231},
  u'LEV': {u'Catalyst-Driven Credit': 0.0,
   u'Deep Value': 0.0,
   u'Merger Related Credit': 20.765900000000002,
   u'Relative Value Credit': 0.0,
   u'Yield to Call': 0.0},
  u'LG': {u'Catalyst-Driven Credit': 14.0216,
   u'Deep Value': 0.0,
   u'Merger Related Credit': 9.8922,
   u'Relative Value Credit': 0.0,
   u'Yield to Call': 0.916},
  u'MACO': {u'Catalyst-Driven Credit': 0.0,
   u'Deep Value': 0.0,
   u'Merger Related Cre

In [14]:
#def get_position_change(start_date_yyyy_mm_dd=None, end_date_yyyy_mm_dd=None, metric="Capital(%)"):
start = time.time()
start_date_yyyy_mm_dd = '2019-04-10'
end_date_yyyy_mm_dd = '2019-03-12'
if end_date_yyyy_mm_dd is None: end_date_yyyy_mm_dd = dbutils.northpoint.get_position_calculated_values_history_max_date()
if start_date_yyyy_mm_dd is None: start_date_yyyy_mm_dd = dbutils.northpoint.get_position_calculated_values_history_second_max_date()

if datetime.datetime.strptime(start_date_yyyy_mm_dd,'%Y-%m-%d') < datetime.datetime(2015,11,9,0,0):
    #return "<h9>Function not supported for dates prior to 11/09/2015</h9>"
    print('the date is too low')

start_pcvh_df = get_position_calculated_values_history(start_date_yyyy_mm_dd,start_date_yyyy_mm_dd)
print(time.time()-start)
start_pcvh_df = start_pcvh_df[start_pcvh_df['Sleeve'] != 'Forwards']

end_pcvh_df = get_position_calculated_values_history(end_date_yyyy_mm_dd,end_date_yyyy_mm_dd)
print(time.time()-start)
end_pcvh_df = end_pcvh_df[end_pcvh_df['Sleeve'] != 'Forwards']

pcvh_df = pd.merge(start_pcvh_df, end_pcvh_df, how='outer',on=["FundCode","Sleeve","Bucket","TradeGroupId","SecurityId"])

pcvh_df = pcvh_df.rename(columns={
        'CatalystName_y':'CatalystName', 'CatalystName_x':'CatalystName[BEFORE]',
        'LongShort_y':'LongShort', 'LongShort_x':'LongShort[BEFORE]',
        'TradeGroup_y':'TradeGroup', 'TradeGroup_x':'TradeGroup[BEFORE]',
        'Capital(%)_y':'Capital(%)', 'Capital(%)_x':'Capital(%)[BEFORE]'
        })

sleeves = list(pcvh_df['Sleeve'].dropna().unique())
fund_codes = list(pcvh_df['FundCode'].dropna().unique())
level_one_tabs = []
for slv in sleeves:
    slv_pcvh_df = pcvh_df[pcvh_df['Sleeve']==slv].copy()
    if len(slv_pcvh_df) == 0: continue

    buckets = slv_pcvh_df['Bucket'].dropna().unique()
    if len(buckets) == 0: continue

    tot_slv_df = pd.DataFrame()
    level_two_tabs = []
    for bucket in buckets:
        bucket_slv_pcvh_df = slv_pcvh_df[slv_pcvh_df['Bucket']==bucket].copy()
        base_cols = ['Bucket','TradeGroup','TradeGroup[BEFORE]','LongShort','LongShort[BEFORE]','CatalystName','CatalystName[BEFORE]']
        #base_cols = ['Bucket','TradeGroup','TradeGroup[BEFORE]','LongShort','CatalystName']
        bucket_slv_pcvh_df[base_cols] = bucket_slv_pcvh_df[base_cols].fillna('')
        df = bucket_slv_pcvh_df[base_cols].drop_duplicates()
        for fund_code in fund_codes:
            fund_capital_cln_df = bucket_slv_pcvh_df[bucket_slv_pcvh_df['FundCode']==fund_code].groupby(base_cols).sum().reset_index()[base_cols+[metric,metric+"[BEFORE]"]].rename(columns={metric:fund_code, metric+"[BEFORE]":fund_code+"[BEFORE]"})
            fund_capital_cln_df[fund_code+'[%Chg]'] = fund_capital_cln_df[fund_code].fillna(0).astype(float) - fund_capital_cln_df[fund_code+'[BEFORE]'].fillna(0).astype(float)
            df = pd.merge(df,fund_capital_cln_df, how='left', on=base_cols)
            df[fund_code+'[%Chg]'] = df[fund_code+'[%Chg]'].fillna('')
            df[fund_code] = [0 if pd.isnull(new) and not pd.isnull(old) else new for (new,old) in zip(df[fund_code],df[fund_code+'[BEFORE]'])]
        tot_slv_df = tot_slv_df.append(df.groupby(['Bucket']).sum().reset_index()[['Bucket']+fund_codes])

        df['TradeGroup'] = [tg if (tg_old == '' or tg==tg_old) else tg_old if tg == '' else tg  + '(previously ' + tg_old+')'  for (tg,tg_old) in zip(df['TradeGroup'],df['TradeGroup[BEFORE]'])]
        df['LongShort'] = [new if (old == '' or new==old) else old if new == '' else new  + '(previously ' + old+')'  for (new,old) in zip(df['LongShort'],df['LongShort[BEFORE]'])]
        df['CatalystName'] = [new if (old == '' or new==old) else old if new == '' else new  + '(previously ' + old+')'  for (new,old) in zip(df['CatalystName'],df['CatalystName[BEFORE]'])]

        del df['Bucket']; del df['TradeGroup[BEFORE]']; del df['LongShort[BEFORE]']; del df['CatalystName[BEFORE]']
        for f in fund_codes: del df[f+'[BEFORE]']
        colnames_to_up_down_arrow = [f+'[%Chg]' for f in fund_codes]
        colnames_to_sum = colnames_to_up_down_arrow + fund_codes
        #level_two_tabs.append((bucket,htmlutils.df2jquery_datatable(df.dropna(axis=1, how='all'),colnames_to_sum=colnames_to_sum,cln2decimal_pts={c:2 for c in colnames_to_sum},up_down_arrow_cls=colnames_to_up_down_arrow)))
    tot_slv_df.index = range(len(tot_slv_df)) # appending messes up index
    tot_slv_df = tot_slv_df.dropna(axis=1, how='all')

    bucket2fund2metric = {bucket:{fund:(float(tot_slv_df[tot_slv_df['Bucket']==bucket][fund].fillna(0).iloc[0]) if fund in tot_slv_df.columns else 0.0) for fund in fund_codes } for bucket in tot_slv_df['Bucket']}

    #slv_tot_chart = highcharts.stacked_bar_chart_negative(
     #   title='Bucket allocations',ylabel=metric,
     #   data=bucket2fund2metric,labels_enabled=True,dec_rounding=2)

    #slv_tot_tab = ('Total',
       #             htmlutils.df2jquery_datatable(tot_slv_df,colnames_to_sum=fund_codes, cln2decimal_pts={f:2 for f in fund_codes})
        #            +"<br><br>" +
        #            slv_tot_chart
        #            )
   # slv_tab_content = htmlutils.tabs_page([slv_tot_tab]+level_two_tabs)
   # level_one_tabs.append((slv,slv_tab_content))

#start_end_date_picker = htmlutils.start_end_date_picker(start_label='As Of',end_label='Compare to', date_format='mm/dd/yy',start_ref_param='as_of',end_ref_param='compare_to')
#return "<br><br>" + start_end_date_picker + "<br><br><center><h9>As of "+end_date_yyyy_mm_dd+" compared to "+start_date_yyyy_mm_dd+"</h9></center><br>" + htmlutils.tabs_page(level_one_tabs)
end = time.time()
print(end-start)

PCVH retrieval completed....
0.663999795914
PCVH retrieval completed....
5.81699991226
8.64699983597


In [13]:
tot_slv_df

,Bucket,CAM,LG,MALT,TACO,AED,ARB,TAQ,LEV,WED,MACO,WIC
0,Conv Trade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.4132,0.0,0.0
